<a href="https://colab.research.google.com/github/olmagra/Proyecto_Predicciones-de-ventas/blob/main/Prediccion_Ventas_Alimentos_M2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PREDICCIONES DE VENTAS ALIMENTICIAS**






In [31]:
# Importar las bibliotecas necesarias:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn import set_config
from sklearn.ensemble import RandomForestRegressor
set_config(display='diagram')

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [32]:
# Código trabajado en ayudantías con el profesor Jesús
#Se requiere esta instrucción para que 'traduzca' la url original en una que entienda Python
def format_drive_url(url):
    file_id = url.split('/')[-2]
    download_url = f'https://drive.google.com/uc?id={file_id}&export=download'
    return download_url

In [33]:
#Cargar los datos desde la url indicada en el proyecto:
url = 'https://drive.google.com/file/d/1syH81TVrbBsdymLT_jl2JIf6IjPXtSQw/view?usp=sharing'
url=format_drive_url(url)
print(url)
df = pd.read_csv(url)
df.head()

https://drive.google.com/uc?id=1syH81TVrbBsdymLT_jl2JIf6IjPXtSQw&export=download


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [34]:
#=> Código tomado del cuaderno de limpieza: https://colab.research.google.com/drive/1zuye5dvjhqZge2rc1tDh_jY3X9FHBzqH#scrollTo=QKiXdVHhNahS
#Eliminar columnas inncesarias para el modelo. Debido a que cada registro tiene un valor único, esto podría meterle ruido al modelo.
df = df.drop(columns=['Item_Identifier'])
df.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [35]:
#=> Código tomado del cuaderno de limpieza: https://colab.research.google.com/drive/1zuye5dvjhqZge2rc1tDh_jY3X9FHBzqH#scrollTo=QKiXdVHhNahS
df.info() #Información sobre el DataFrame:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Weight                7060 non-null   float64
 1   Item_Fat_Content           8523 non-null   object 
 2   Item_Visibility            8523 non-null   float64
 3   Item_Type                  8523 non-null   object 
 4   Item_MRP                   8523 non-null   float64
 5   Outlet_Identifier          8523 non-null   object 
 6   Outlet_Establishment_Year  8523 non-null   int64  
 7   Outlet_Size                6113 non-null   object 
 8   Outlet_Location_Type       8523 non-null   object 
 9   Outlet_Type                8523 non-null   object 
 10  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(6)
memory usage: 732.6+ KB


In [36]:
# Cambiar el tipo de dato de la columna 'Outlet_Establishment_Year' from int to an object
df['Outlet_Establishment_Year'] = df['Outlet_Establishment_Year'].astype(object)
df.info() #Información sobre el DataFrame:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Weight                7060 non-null   float64
 1   Item_Fat_Content           8523 non-null   object 
 2   Item_Visibility            8523 non-null   float64
 3   Item_Type                  8523 non-null   object 
 4   Item_MRP                   8523 non-null   float64
 5   Outlet_Identifier          8523 non-null   object 
 6   Outlet_Establishment_Year  8523 non-null   object 
 7   Outlet_Size                6113 non-null   object 
 8   Outlet_Location_Type       8523 non-null   object 
 9   Outlet_Type                8523 non-null   object 
 10  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), object(7)
memory usage: 732.6+ KB


In [37]:
#=> Código tomado del cuaderno de limpieza: https://colab.research.google.com/drive/1zuye5dvjhqZge2rc1tDh_jY3X9FHBzqH#scrollTo=QKiXdVHhNahS
df.duplicated().sum() #Buscar y sumar duplicados

0

In [38]:
#=> Código tomado del cuaderno de limpieza: https://colab.research.google.com/drive/1zuye5dvjhqZge2rc1tDh_jY3X9FHBzqH#scrollTo=QKiXdVHhNahS
df.isna().sum() #Identifica los valores faltantes por columna 

Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

Identifiquen el objetivo (X) y las características (y): Asignen la columna "Item_Outlet_Sales" somo el objetivo y el resto de las variables relevantes como el matriz de características.  

In [39]:
#=> Código tomado de lección: https://login.codingdojo.com/m/503/12406/86754
X = df.drop(columns = 'Item_Outlet_Sales')
y = df['Item_Outlet_Sales']

Realicen un train test split 

In [40]:
#=> Código tomado de lección: https://login.codingdojo.com/m/503/12406/86754
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

Creen un pipeline de preprocesamiento para preparar el conjunto de datos para el aprendizaje automático.

Identificar a qué columnas le faltan valores y decidir qué estrategia de imputación usar para rellenarlas

In [41]:
#=> Código tomado de lección 'Simple Imputer' https://login.codingdojo.com/m/503/12406/86756
print(df.isna().sum().sum(), 'missing values') #Resumir # de valores faltantes

3873 missing values


Convertir el año en categórico

In [42]:
# instancien los selectores a nuestros tipos de datos numéricos y categóricos
num_selector = make_column_selector(dtype_include='number')
cat_selector = make_column_selector(dtype_include='object')

# seleccionen las columnas numéricas de cada tipo
num_columns = num_selector(X_train)
cat_columns = cat_selector(X_train)

# comprueben las listas
print('Columnas numéricas:', num_columns)
print('Columnas categóricas:', cat_columns)

Columnas numéricas: ['Item_Weight', 'Item_Visibility', 'Item_MRP']
Columnas categóricas: ['Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']


In [43]:
#=> Código tomado de lección Coding Dojo: https://login.codingdojo.com/m/503/12406/86756
X_train.isna().any() #comprobar a qué columnas le faltan datos

Item_Weight                   True
Item_Fat_Content             False
Item_Visibility              False
Item_Type                    False
Item_MRP                     False
Outlet_Identifier            False
Outlet_Establishment_Year    False
Outlet_Size                   True
Outlet_Location_Type         False
Outlet_Type                  False
dtype: bool

In [44]:
#=> Código tomado de lección Coding Dojo: https://login.codingdojo.com/m/503/12406/86756
# aíslen las columnas numéricas: A la columna 'Item_Weight' le faltan datos
df_num = df[num_columns]
# aíslen las columnas con datos faltantes
df_num.loc[:, df_num.isna().any()]

# La columna 'Item_Weight' tiene valores flotantes, por lo tanto se elige la estrategia de 'mean' (promedio)

,Item_Weight
0,9.300
1,5.920
2,17.500
3,19.200
4,8.930
...,...
8518,6.865
8519,8.380
8520,10.600
8521,7.210


In [45]:
#=> Código tomado de Lección Coding Dojo: https://login.codingdojo.com/m/503/12406/86756
# Instancien el objeto imputer de la clase SimpleImputer con la estrategia 'mean' (promedio)
mean_imputer = SimpleImputer(strategy='mean')
# Encajen el objeto imputer en los datos de entrenamiento numérico con .fit() 
# calculen 'mean' (promedio) de las columnas en el conjunto de entrenamiento
mean_imputer.fit(X_train[num_columns])
#Utilicen 'mean' (promedio) a partir de los datos de entrenamiento para rellenar los valores que en falten
#las columnas numericas de los conjuntos de entrenamiento y de prueba con  .transform()
X_train.loc[:, num_columns] = mean_imputer.transform(X_train[num_columns])
X_test.loc[:, num_columns] = mean_imputer.transform(X_test[num_columns])


In [46]:
#=> Código tomado de Lección Coding Dojo: https://login.codingdojo.com/m/503/12406/86756
# Verificar si SimpleImputer rellenó los valores faltantes en X_train
X_train.isna().any()

Item_Weight                  False
Item_Fat_Content             False
Item_Visibility              False
Item_Type                    False
Item_MRP                     False
Outlet_Identifier            False
Outlet_Establishment_Year    False
Outlet_Size                   True
Outlet_Location_Type         False
Outlet_Type                  False
dtype: bool

In [47]:
#=> Código tomado de Lección Coding Dojo: https://login.codingdojo.com/m/503/12406/86756
# Rellenar los valores faltantes de la columna 'Outlet_Size'
# Instancien el objeto imputer de la clase SimpleImputer con la estrategia 'constant' (valor constante)
constant_imputer = SimpleImputer(strategy='constant', fill_value='Small')

# Encajen el objeto imputer en los datos de entrenamiento numérico con .fit() 
# calculen 'constant' (valor constante) de las columnas en el conjunto de entrenamiento
constant_imputer.fit(X_train[cat_columns])

#Utilicen 'constant' (valor constante) a partir de los datos de entrenamiento para rellenar los valores que en falten
#las columnas categóricas de los conjuntos de entrenamiento y de prueba con  .transform()
# Se reemplazan los valores nulos por correctos
X_train.loc[:, cat_columns] = constant_imputer.transform(X_train[cat_columns])
X_test.loc[:, cat_columns] = constant_imputer.transform(X_test[cat_columns])

print(X_train)
print(X_test)

      Item_Weight Item_Fat_Content  Item_Visibility              Item_Type  \
4776       16.350          Low Fat         0.029565              Household   
7510       15.250          Regular         0.000000            Snack Foods   
5828       12.350          Regular         0.158716                   Meat   
5327        7.975          Low Fat         0.014628           Baking Goods   
4810       19.350          Low Fat         0.016645           Frozen Foods   
...           ...              ...              ...                    ...   
5734        9.395          Regular         0.286345  Fruits and Vegetables   
5191       15.600          Low Fat         0.117575           Frozen Foods   
5390       17.600          Low Fat         0.018944     Health and Hygiene   
860        20.350          low fat         0.054363            Snack Foods   
7270       16.350               LF         0.016993              Household   

      Item_MRP Outlet_Identifier  Outlet_Establishment_Year Out

<ipython-input-47-69ed893aa630>:13: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_train.loc[:, cat_columns] = constant_imputer.transform(X_train[cat_columns])
<ipython-input-47-69ed893aa630>:14: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_test.loc[:, cat_columns] = constant_imputer.transform(X_test[cat_columns])


In [48]:
#=> Código tomado de Lección Coding Dojo: https://login.codingdojo.com/m/503/12406/86756
# Verificar si SimpleImputer rellenó los valores faltantes en X_train
X_train.isna().any()

Item_Weight                  False
Item_Fat_Content             False
Item_Visibility              False
Item_Type                    False
Item_MRP                     False
Outlet_Identifier            False
Outlet_Establishment_Year    False
Outlet_Size                  False
Outlet_Location_Type         False
Outlet_Type                  False
dtype: bool

Convertir en ordinal el tamaño de Outlet

In [49]:
df['Outlet_Size'].value_counts() #Cuento valores para revisar qué valores tiene este campo categórico

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64

In [50]:
#=> Cödigo tomado de lección Coding Dojo: https://login.codingdojo.com/m/503/12406/86750
# Reemplazar las características ordinales del campo 'Outlet_Size' con números
# asignen el objetivo y, y las características X
outlet_sizes = {'Small': 1, 'Medium': 2, 'High': 3}

# definan diccionario para reemplazar
X_train['Outlet_Size'] = X_train['Outlet_Size'].replace(outlet_sizes)

# apliquen el diccionario a la columna en el conjunto de entrenamiento
X_test['Outlet_Size'] = X_test['Outlet_Size'].replace(outlet_sizes)

#Revisar el DataFrame para asegurarnos que funciona:
X_train.head()



,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
4776,16.350,Low Fat,0.029565,Household,256.4646,OUT018,2009,2,Tier 3,Supermarket Type2
7510,15.250,Regular,0.000000,Snack Foods,179.7660,OUT018,2009,2,Tier 3,Supermarket Type2
5828,12.350,Regular,0.158716,Meat,157.2946,OUT049,1999,2,Tier 1,Supermarket Type1
5327,7.975,Low Fat,0.014628,Baking Goods,82.3250,OUT035,2004,1,Tier 2,Supermarket Type1
4810,19.350,Low Fat,0.016645,Frozen Foods,120.9098,OUT045,2002,1,Tier 2,Supermarket Type1


In [51]:
# Realizar una codificación one-hot a las características nominales
#=> Código tomado de lección: https://login.codingdojo.com/m/503/12406/86754
ohe = OneHotEncoder(handle_unknown='ignore') #codificar las columnas categóricas

In [52]:
#=> Código tomado de lección: https://login.codingdojo.com/m/503/12406/86754
scaler = StandardScaler() # Utilizar StandardScaler para escalar las columnas numéricas

In [53]:
#=> Código tomado de lección: https://login.codingdojo.com/m/503/12406/86754
# Utilizar un ColumnTransformer para emparejar la transformación al tipo de columna
# Emparejar las columnas con la transformación que queremos aplicar:
# Hacer Tuplas para preprocesar las columnas categóricas y numéricas
num_tuple = (scaler, num_selector)
cat_tuple = (ohe, cat_selector)

In [54]:
#=> Código tomado de lección: https://login.codingdojo.com/m/503/12406/86754
# Ahora tenemos de instanciar el transformador de columna y agregar cada una de las tuplas que creamos antes
from sklearn.compose import make_column_transformer
col_transformer = make_column_transformer(num_tuple, cat_tuple, remainder = 'passthrough')

In [55]:
#=> Código tomado de lección: https://login.codingdojo.com/m/503/12406/86754
# Encajar el transformador
col_transformer.fit(X_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('standardscaler', StandardScaler(),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fa6144279a0>),
                                ('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fa614427580>)])

In [56]:
# Ahora podemos transformar los conjuntos de entrenamiento y de prueba
X_train_processed = col_transformer.transform(X_train)
X_test_processed = col_transformer.transform(X_test)

# Ver las transformaciones:
X_train_df = pd.DataFrame(X_train_processed.toarray())
X_train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,0.817249,-0.712775,1.828109,1.327849,0.668862,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.556340,-1.291052,0.603369,1.327849,0.668862,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,-0.131512,1.813319,0.244541,0.136187,0.668862,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,-1.169219,-1.004931,-0.952591,0.732018,-0.799831,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,1.528819,-0.965484,-0.336460,0.493686,-0.799831,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [57]:
X_train_df.columns = col_transformer.get_feature_names_out()
X_train_df

,standardscaler__Item_Weight,standardscaler__Item_Visibility,standardscaler__Item_MRP,standardscaler__Outlet_Establishment_Year,standardscaler__Outlet_Size,onehotencoder__Item_Fat_Content_LF,onehotencoder__Item_Fat_Content_Low Fat,onehotencoder__Item_Fat_Content_Regular,onehotencoder__Item_Fat_Content_low fat,onehotencoder__Item_Fat_Content_reg,...,onehotencoder__Outlet_Identifier_OUT045,onehotencoder__Outlet_Identifier_OUT046,onehotencoder__Outlet_Identifier_OUT049,onehotencoder__Outlet_Location_Type_Tier 1,onehotencoder__Outlet_Location_Type_Tier 2,onehotencoder__Outlet_Location_Type_Tier 3,onehotencoder__Outlet_Type_Grocery Store,onehotencoder__Outlet_Type_Supermarket Type1,onehotencoder__Outlet_Type_Supermarket Type2,onehotencoder__Outlet_Type_Supermarket Type3
0,0.817249,-0.712775,1.828109,1.327849,0.668862,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.556340,-1.291052,0.603369,1.327849,0.668862,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,-0.131512,1.813319,0.244541,0.136187,0.668862,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,-1.169219,-1.004931,-0.952591,0.732018,-0.799831,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,1.528819,-0.965484,-0.336460,0.493686,-0.799831,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6387,-0.832409,4.309657,-0.044657,0.017021,-0.799831,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
6388,0.639356,1.008625,-1.058907,1.089517,-0.799831,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
6389,1.113736,-0.920527,1.523027,0.493686,-0.799831,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
6390,1.766009,-0.227755,-0.383777,1.089517,-0.799831,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [58]:
X_train_df.info() # Ver la info del DataFrame

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6392 entries, 0 to 6391
Data columns (total 43 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   standardscaler__Item_Weight                     6392 non-null   float64
 1   standardscaler__Item_Visibility                 6392 non-null   float64
 2   standardscaler__Item_MRP                        6392 non-null   float64
 3   standardscaler__Outlet_Establishment_Year       6392 non-null   float64
 4   standardscaler__Outlet_Size                     6392 non-null   float64
 5   onehotencoder__Item_Fat_Content_LF              6392 non-null   float64
 6   onehotencoder__Item_Fat_Content_Low Fat         6392 non-null   float64
 7   onehotencoder__Item_Fat_Content_Regular         6392 non-null   float64
 8   onehotencoder__Item_Fat_Content_low fat         6392 non-null   float64
 9   onehotencoder__Item_Fat_Content_reg      